In [1]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

from stockfish import Stockfish

import torch

from langchain import HuggingFacePipeline, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
#model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model_id = "meta-llama/Llama-2-13b-chat-hf"
model_id = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            load_in_4bit=True,
                                            use_cache=False, 
                                            device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 8/8 [00:10<00:00,  1.37s/it]


In [4]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.1,
    #top_p=0.15,
    #repetition_penalty=1
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
def init_json_chat_history():
    json_chat_history = {"Opponent": [], "Carlus Magnusen": []}

    return json_chat_history

In [6]:
def build_chat_history(message, json_chat_history, bot):

    if bot == 0:
        json_chat_history["Opponent"].append(message)
    elif bot == 1:
        json_chat_history["Carlus Magnusen"].append(message)

    formatted_dialogue = []

    for i in range(len(json_chat_history["Opponent"])):
        formatted_dialogue.append(f'Opponent: {json_chat_history["Opponent"][i]}')
        if i < len(json_chat_history["Carlus Magnusen"]):
            formatted_dialogue.append(f'Carlus Magnusen: {json_chat_history["Carlus Magnusen"][i]}')

    formatted_chat_history = '\n'.join(formatted_dialogue)

    return json_chat_history, formatted_chat_history

In [7]:
def build_prompt(prompt, formatted_chat_history, tool):

#     template = f"""[INST] <<SYS>>
# You are a helpful, respectful and honest assistant named Carlus Magnusen.
# Act in a professional manner and keep your answers short.
# Bellow you will find the best chess move, you should tell your opponent that this is the move you play.
# Best chess move: e6
# <</SYS>>
# {formatted_chat_history}[/INST] 
# """

#     template = f"""[INST] <<SYS>>
# Your objective is to select a tool based on the user input.
# There are two possible tools: [chat, chess]
# Select the chess tool only if the user makes a chess move and select the chat tool otherwise
# You must give your answer in the following json format:
# {{"tool": '<<tool that you chose>>'}}
# <</SYS>>
# {prompt}[/INST]
# """

    if tool == "classifier":
        sys_message = """[INST] <<SYS>>
Your objective is to classify the user input.
There are two possible calssifications: [chess move, no chess move]
If you the user plays a chess move against you, your classification should be "chess move".
If you the user does not play a chess move against you, your classification should be "no chess move".
You must give your answer in the following json format:
{"classification": "<<classification you chose>>"}
<</SYS>>
"""

        full_prompt = sys_message + f"""{prompt}""" + "[/INST]\n"

        sys_message = """<s>[INST] Your objective is to classify the user input.
There are two possible calssifications: [chess move, no chess move]
If you the user plays a chess move against you, your classification should be "chess move".
If you the user does not play a chess move against you, your classification should be "no chess move".
You must give your answer in the following json format:
{"classification": "<<classification you chose>>"}
"""

        full_prompt = sys_message + f"""{prompt}""" + " [/INST] "

    if tool == "chat_bot":
        template = f"""[INST] <<SYS>>
You are a helpful, respectful and honest assistant named Carlus Magnusen.
Act in a professional manner and keep your answers short.
# <</SYS>>
# {formatted_chat_history}[/INST]
"""

        full_prompt = sys_message + f"""{prompt}""" + "[/INST]\n"

    return full_prompt

In [8]:
json_chat_history = init_json_chat_history()

In [9]:
prompt = "who are you"
json_chat_history, formatted_chat_history = build_chat_history(prompt, json_chat_history, bot=0)

# prompt = "im carlus"
# json_chat_history, formatted_chat_history = build_chat_history(prompt, json_chat_history, bot=1)

# prompt = "hi carlus"
# json_chat_history, formatted_chat_history = build_chat_history(prompt, json_chat_history, bot=0)

full_prompt = build_prompt(prompt, formatted_chat_history, "classifier")

In [10]:
print(full_prompt)

[INST] <<SYS>>
Your objective is to classify the user input.
There are two possible calssifications: [chess move, no chess move]
If you the user plays a chess move against you, your classification should be "chess move".
If you the user does not play a chess move against you, your classification should be "no chess move".
You must give your answer in the following json format:
{"classification": "<<classification you chose>>"}
<</SYS>>
who are you[/INST]



In [11]:
output = local_llm(full_prompt)

json_chat_history, formatted_chat_history = build_chat_history(output, json_chat_history, bot=1)

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(output)

{"tool": "chat"}


In [18]:
json_chat_history = init_json_chat_history()
prompt = "I play my bishop, just not sure where"
json_chat_history, formatted_chat_history = build_chat_history(prompt, json_chat_history, bot=0)
full_prompt = build_prompt(prompt, formatted_chat_history, "classifier")
output = local_llm(full_prompt)
json_chat_history, formatted_chat_history = build_chat_history(output, json_chat_history, bot=1)

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [20]:
print(formatted_chat_history)

Opponent: I play my bishop, just not sure where
Carlus Magnusen: 
{"classification": "chess move"}


In [68]:
print(formatted_chat_history)

Opponent: I play my bishop, just not sure where
Carlus Magnusen: {"classification": "chess move"}


In [16]:
prompt = "hello"

In [23]:
chat_history, formatted_chat_history = build_chat_history(prompt, chat_history, bot=0)

template = build_prompt(prompt, formatted_chat_history)

prompt = PromptTemplate(
    #input_variables=["human_input", "history"], 
    input_variables=["human_input"], 
    template=template
)

simple_chain = LLMChain(
    llm=local_llm, 
    prompt=prompt, 
    verbose=True, 
    #memory=ConversationBufferWindowMemory(k=2),
)

output = simple_chain.predict(human_input=prompt)

chat_history, formatted_chat_history = build_chat_history(output, chat_history, bot=1)

ValueError: Missing some input keys: {'formatted_chat_history', 'prompt'}

In [84]:
chat_history

{'Opponent': ["I'm Carlus Magnusen"],
 'Carlus Magnusen': ["I'm Carlus Magnusen", "I'm Carlus Magnusen"]}

In [86]:
print(formatted_chat_history)

Opponent: I'm Carlus Magnusen
Carlus Magnusen: I'm Carlus Magnusen
Carlus Magnusen: I'm Carlus Magnusen


In [ ]:
def build_prompt(prompt, chat_history):

formated_chat_history = """
"""

    template = """[INST] <<SYS>>
You are a Carlus Magnusen, the best chess player in the known universe and your purpose is to play chess with whoever wants to play with you. You are very confident and arrogant. You trash talk your opponents when you play chess. Bellow you will find the best move, you should always tell your opponent that this is the move you play while taunting him.
Best Move: e6
<</SYS>>
{chat_history}
Opponent: {prompt}[/INST]
Carlus Magnusen: 
"""

return template

In [44]:
chat_history = {}

In [46]:
chat_history = prompt_chatbot("hi", chat_history)

In [47]:
chat_history

{'Opponent': ['hi', 'hi']}

In [ ]:
if "Opponent" not in chat_history:
    chat_history["Opponent"] = "Hello, how can I help you?"
else:
    chat_history["Opponent"] = "I need assistance with a different issue."

In [ ]:
sf_config = {
    "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 8, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 2048, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 10,
    "Minimum Thinking Time": 20,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 9999
}

Stockfish(path="/mnt/c/Users/Antonio/Desktop/Projects/chess_bot/stockfish/stockfish-windows-x86-64-avx2.exe",  depth=22, parameters=sf_config)